#### Original version of findCenter()
This function evaluates the circumcenter of the `P` points.If the points lies on a `d-1` circumball then the function is not able to perform the evaluation and therefore returns a `NaN` array.


In [ ]:
function findCenter(P::Lar.Points)::Array{Float64,1}
    dim, n = size(P)
    @assert n > 0		"findCenter: at least one points is needed."
    @assert dim >= n-1	"findCenter: Too much points"
    @assert dim < 4		"findCenter: Function not yet Programmed."

    if n == 1
        center = P[:, 1]

    elseif n == 2
        #for each dimension
        center = (P[:, 1] + P[:, 2]) / 2

    elseif n == 3
        #https://www.ics.uci.edu/~eppstein/junkyard/circumcenter.html
        if dim == 2
            denom = 2 * Lar.det([ P[:, 2] - P[:, 1]  P[:, 3] - P[:, 1] ])
            deter = (P[:, 2] - P[:, 1]) * Lar.norm(P[:, 3] - P[:, 1])^2 -
                    (P[:, 3] - P[:, 1]) * Lar.norm(P[:, 2] - P[:, 1])^2
            numer = [- deter[2], deter[1]]
            center = P[:, 1] + numer / denom

        elseif dim == 3
            #circumcenter of a triangle in R^3
            numer = Lar.norm(P[:, 3] - P[:, 1])^2 * Lar.cross(
                        Lar.cross(P[:, 2] - P[:, 1], P[:, 3] - P[:, 1]),
                        P[:, 2] - P[:, 1]
                    ) +
                    Lar.norm(P[:, 2] - P[:, 1])^2 * Lar.cross(
                        P[:, 3] - P[:, 1],
                        Lar.cross(P[:, 2] - P[:, 1], P[:, 3] - P[:, 1]
                    )
            )
            denom = 2 * Lar.norm(
                Lar.cross(P[:, 2] - P[:, 1], P[:, 3] - P[:, 1])
            )^2
            center = P[:, 1] + numer / denom
        end

    elseif n == 4 #&& dim = 3
        # https://people.sc.fsu.edu/~jburkardt/presentations
        #	/cg_lab_tetrahedrons.pdf
        # page 6 (matrix are transposed)
        α = Lar.det([P; ones(1, 4)])
        sq = sum(abs2, P, dims = 1)
        Dx = Lar.det([sq; P[2:2,:]; P[3:3,:]; ones(1, 4)])
        Dy = Lar.det([P[1:1,:]; sq; P[3:3,:]; ones(1, 4)])
        Dz = Lar.det([P[1:1,:]; P[2:2,:]; sq; ones(1, 4)])
        center = [Dx; Dy; Dz]/2α
    end

    return center
#	AlphaStructures.foundCenter([P[:,i] for i = 1 : size(P, 2)])[:,:]
end


#### Modified version
For parallelize this function, we subdivided it to four functions, as shown below:

In [ ]:

# funzione ausiliaria per findcenter
# calcola il secondo membro del numeratore se dim = 3
function secondMember3d(P::Lar.Points)::Array{Float64,1}
     n2 =  Lar.norm(P[:, 2] - P[:, 1])^2 * Lar.cross(
       P[:, 3] - P[:, 1],
       Lar.cross(P[:, 2] - P[:, 1], P[:, 3] - P[:, 1]
        ))
    return n2
end

# funzione ausiliaria per findcenter
# calcola il primo membro del numeratore se dim = 3
function firstMember3d(P::Lar.Points)::Array{Float64,1}
    n1 =  Lar.norm(P[:, 3] - P[:, 1])^2 * Lar.cross(
                Lar.cross(P[:, 2] - P[:, 1], P[:, 3] - P[:, 1]),
                P[:, 2] - P[:, 1]
            )
    return n1
end

# funzione ausiliaria per findcenter
# calcola il denominatore se dim = 3
function denominatore3d(P::Lar.Points)::Float64
    d =	2 * ( Lar.norm(
            Lar.cross(P[:, 2] - P[:, 1], P[:, 3] - P[:, 1])))^2
    return d
end


function deter2d(P::Lar.Points)::Array{Float64,1}
    return (P[:, 2] - P[:, 1]) * Lar.norm(P[:, 3] - P[:, 1])^2 -
            (P[:, 3] - P[:, 1]) * Lar.norm(P[:, 2] - P[:, 1])^2
end

function denom2d(P::Lar.Points)::Float64
    return 2 * Lar.det([ P[:, 2] - P[:, 1]  P[:, 3] - P[:, 1] ])
end


In [ ]:
@timeit to "findCenter" function findCenter(P::Lar.Points)::Array{Float64,1}
    dim, n = size(P)
    @assert n > 0		"findCenter: at least one points is needed."
    @assert dim >= n-1	"findCenter: Too much points"
    @assert dim < 4		"findCenter: Function not yet Programmed."

    if n == 1
        center =  P[:, 1]

    elseif n == 2
        #for each dimension
        center =  (P[:, 1] + P[:, 2]) / 2

    elseif n == 3
        #https://www.ics.uci.edu/~eppstein/junkyard/circumcenter.html
        if dim == 2
            denom = denom2d(P)
            deter = deter2d(P)

            numer = [- deter[2], deter[1]]
            center = P[:, 1] + numer / denom

        elseif dim == 3
            #circumcenter of a triangle in R^3
            n1 = firstMember3d(P)
            n2 = secondMember3d(P)

            numer = n1+n2
            denom = denominatore3d(P)
            center = P[:, 1] + numer / denom
        end

    elseif n == 4 #&& dim = 3
        # https://people.sc.fsu.edu/~jburkardt/presentations
        #	/cg_lab_tetrahedrons.pdf
        # page 6 (matrix are transposed)

        α = Lar.det([P; ones(1, 4)])
        sq = sum(abs2, P, dims = 1)
        Dx =  Lar.det([sq; P[2:2,:]; P[3:3,:]; ones(1, 4)])
        Dy = Lar.det([P[1:1,:]; sq; P[3:3,:]; ones(1, 4)])
        Dz = Lar.det([P[1:1,:]; P[2:2,:]; sq; ones(1, 4)])
        center = [Dx; Dy; Dz]/2α
    end

    return center
#	AlphaStructures.foundCenter([P[:,i] for i = 1 : size(P, 2)])[:,:]
end


#### Original version of oppositeHalfSpacePoints()
This function returns the index list of the points `P` located in the halfspace defined by`face` points that do not contains the point `point`.
_Obs._ Dimension Dipendent, only works if dimension is three or less and
	the number of points in the face is the same than the dimension.

In [ ]:
function oppositeHalfSpacePoints(
        P::Lar.Points,
        face::Array{Float64,2},
        point::Array{Float64,1}
    )::Array{Int64,1}

    dim, n = size(P)
    noV = size(face, 2)
    @assert dim <= 3 "oppositeHalfSpacePoints: Not yet coded."
    @assert noV == dim "oppositeHalfSpacePoints:
        Cannot determine opposite to non hyperplanes."
    if dim == 1
        threshold = face[1]
        if point[1] < threshold
            opposite = [i for i = 1 : n if P[1, i] > threshold]
        else
            opposite = [i for i = 1 : n if P[1, i] < threshold]
        end
    elseif dim == 2
        if (Δx = face[1, 1] - face[1, 2]) != 0.0
            m = (face[2, 1] - face[2, 2]) / Δx
            q = face[2, 1] - m * face[1, 1]
            # false = under the line, true = over the line
            @assert point[2] ≠ m * point[1] + q "oppositeHalfSpacePoints,
                the point belongs to the face"
            side = sign(m * point[1] + q - point[2])
            opposite =
                [i for i = 1 : n if side * (m * P[1, i] + q - P[2, i]) < 0]
        else
            q = face[1, 1]
            side = sign(point[1] - q)
            opposite = [i for i = 1 : n if side * (P[1, i] - q) < 0]
        end


    elseif dim == 3
        axis = Lar.cross(
            face[:, 2] - face[:, 1],
            face[:, 3] - face[:, 1]
        )
        off = Lar.dot(axis, face[:, 1])
        position = Lar.dot(point, axis)
        if position < off
            opposite = [i for i = 1:size(P, 2) if Lar.dot(P[:,i], axis) > off]
        else
            opposite = [i for i = 1:size(P, 2) if Lar.dot(P[:,i], axis) < off]
        end
    end

    return [
        i for i in opposite
        if sum([P[:, i] == face[:, j] for j = 1 : noV]) == 0
    ]

end

#### Modified version
For parallelize this function, we used @simd

In [ ]:
@timeit to "oppositeHalfSpacePoints" function oppositeHalfSpacePoints(
        P::Lar.Points,
        face::Array{Float64,2},
        point::Array{Float64,1}
    )::Array{Int64,1}


    dim, n = size(P)
    noV = size(face, 2)
    @assert dim <= 3 "oppositeHalfSpacePoints: Not yet coded."
    @assert noV == dim "oppositeHalfSpacePoints:
        Cannot determine opposite to non hyperplanes."
    opposite = Array{Int64,1}()

    if dim == 1
        threshold = face[1]
        if point[1] < threshold
            #Per parallelizzare il metodo, abbiamo trasformato questo codice nel
            #codice che segue
            @inbounds @simd for i=1 : n
                if P[1,i] > threshold
                    push!(opposite,i)
                end
            end

        else
            #Per parallelizzare il metodo, abbiamo trasformato questo codice nel
            #codice che segue
            @inbounds @simd for i =1 : n
                if P[1,i]< threshold
                    push!(opposite,i)
                end
            end

        end
    elseif dim == 2
        if (Δx = face[1, 1] - face[1, 2]) != 0.0
            m = (face[2, 1] - face[2, 2]) / Δx
            q = face[2, 1] - m * face[1, 1]
            # false = under the line, true = over the line
            @assert point[2] ≠ m * point[1] + q "oppositeHalfSpacePoints,
                the point belongs to the face"
            side = sign(m * point[1] + q - point[2])
            #Per parallelizzare il metodo, abbiamo trasformato questo codice nel
            #codice che segue

            @inbounds @simd for i=1 : n
                if side * (m * P[1, i] + q - P[2, i]) < 0
                    push!(opposite,i)
                end
            end

        else
            q = face[1, 1]
            side = sign(point[1] - q)

             @inbounds @simd for i = 1 : n
                if side * (P[1, i] - q) < 0
                    push!(opposite,i)
                end
            end

        end

    elseif dim == 3

        axis =  Lar.cross(
            face[:, 2] - face[:, 1],
            face[:, 3] - face[:, 1]
        )
        off =  Lar.dot(axis, face[:, 1])
        position =  Lar.dot(point, axis)

        #Per parallelizzare il metodo, abbiamo trasformato questo codice nel
        #codice che segue
        if position < off
            @inbounds @simd for i=1:size(P,2)
                if Lar.dot(P[:,i], axis) > off
                    push!(opposite,i)
                end
            end
        else
            @inbounds @simd for i = 1:size(P, 2)
                if Lar.dot(P[:,i], axis) < off
                    push!(opposite,i)
                end
            end
        end
    end

    return [
        i for i in opposite
        if sum([P[:, i] == face[:, j] for j = 1 : noV]) == 0
    ]
end

#### Original version of planarIntersection

In [ ]:
@timeit to "planarIntersection" function planarIntersection(
        P::Lar.Points,
        face::Array{Int64,1},
        axis::Int64,
        off::Float64
    )::Int64

    pos = [P[axis, i] > off for i in face]

    if sum([P[axis, i] == off for i in face]) == length(pos)
        position = 0
    elseif sum(pos) == 0
        position = -1
    elseif sum(pos) == length(pos)
        position = +1
    else
        position = 0
    end

    return position
end



#### Modified version of planarIntersection

In [ ]:
@timeit to "planarIntersection" function planarIntersection(
        P::Lar.Points,
        face::Array{Int64,1},
        axis::Int64,
        off::Float64
    )::Int64
    #per paralellizzare il metodo abbiamo trasformato questo codice nel
    #codice che segue


    pos= Array{Int64,1}()
    @inbounds @simd for i in face
        if P[axis,i] > off
            push!(pos,1)
        else
            push!(pos,0)
        end
    end

    if sum([P[axis, i] == off for i in face]) == length(pos)
        position = 0
    elseif sum(pos) == 0
        position = -1
    elseif sum(pos) == length(pos)
        position = +1
    else
        position = 0
    end

    return position
end
